# covariates , tagsを作成する

In [1]:
# すでにcomplete_tags,complete_death_tagsがあるのでUNIQUE_IDとsample_idを合わせる
# sapsiiもsample idをつけたデータセットとして必要列のみ残したデータセットを作成しておく

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import datetime
from datetime import timedelta
import numpy as np

from tqdm import tqdm
import time

file_path="D:/mimic_iii/csv/"
outfile_path="D:/mimic_iii/clean_data/"

In [3]:
df=pd.read_csv(outfile_path+'GRU_ODE_Dataset.csv')

In [4]:
tags=pd.read_csv(outfile_path+"complete_death_tags.csv")
covs=pd.read_csv(outfile_path+"complete_covariates.csv")
saps=pd.read_csv(file_path+"sapsii_processed.csv")
id_ref=pd.read_csv(outfile_path+"UNIQUE_ID_dict.csv")

In [5]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [6]:
# ロード
d=pickle_load( outfile_path+'sample_unique_id_dict.pickle')

In [7]:
# sample_id->unique_idをunique_id->sample_idに
# 
def get_swap_dict(d):
    return {v: k for k, v in d.items()}

d_swap = get_swap_dict(d)
print(d_swap)

 2530: 19769, 11451: 19770, 10303: 19771, 7682: 19772, 12252: 19773, 12384: 19774, 12364: 19775, 9195: 19776, 10489: 19777, 13414: 19778, 8931: 19779, 14485: 19780, 11690: 19781, 12477: 19782, 18879: 19783, 2339: 19784, 11367: 19785, 15058: 19786, 12002: 19787, 14772: 19788, 17204: 19789, 3809: 19790, 1915: 19791, 4608: 19792, 7905: 19793, 17223: 19794, 9060: 19795, 9524: 19796, 5138: 19797, 19885: 19798, 3901: 19799, 20035: 19800, 5449: 19801, 8695: 19802, 3713: 19803, 17980: 19804, 4206: 19805, 12528: 19806, 513: 19807, 396: 19808, 13851: 19809, 20944: 19810, 19623: 19811, 15853: 19812, 5479: 19813, 5091: 19814, 4534: 19815, 12981: 19816, 8853: 19817, 5164: 19818, 47: 19819, 4992: 19820, 11044: 19821, 3283: 19822, 9008: 19823, 13613: 19824, 2295: 19825, 1188: 19826, 15226: 19827, 11461: 19828, 11798: 19829, 12937: 19830, 11707: 19831, 7558: 19832, 1821: 19833, 13628: 19834, 8785: 19835, 9055: 19836, 18118: 19837, 10947: 19838, 19294: 19839, 11168: 19840, 20857: 19841, 15419: 19842, 3

# tags process

In [8]:
tags

,Unnamed: 0,HADM_ID,Value,UNIQUE_ID
0,6220,126509,0,0
1,11659,149789,0,1
2,5313,122678,0,2
3,17838,176370,0,3
4,6475,127622,0,4
...,...,...,...,...
21245,9127,138935,0,21245
21246,5594,123826,0,21246
21247,12443,153228,1,21247
21248,2152,109260,0,21248


In [9]:
hadm_uniq = tags.loc[:,["HADM_ID","UNIQUE_ID"]]
hadm_uniq.head()

,HADM_ID,UNIQUE_ID
0,126509,0
1,149789,1
2,122678,2
3,176370,3
4,127622,4


In [10]:
hadm_uniq["HADM_ID"].nunique(), hadm_uniq["UNIQUE_ID"].nunique(), hadm_uniq.shape

(21250, 21250, (21250, 2))

UNIQUE_IDはDataMergingの段階で振られたHADM‗IDと一対一対応するIDである。その対応表は、clean_data/UNIQUE_ID_dict.csvに保存されている。


In [11]:
if "Unnamed: 0" in tags.columns:
    tags=tags.drop("Unnamed: 0",axis=1)
if "HADM_ID" in tags.columns:
    tags=tags.drop("HADM_ID",axis=1)
tags

,Value,UNIQUE_ID
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
21245,0,21245
21246,0,21246
21247,1,21247
21248,0,21248


In [12]:
tags=tags.reindex(columns=['UNIQUE_ID', 'Value'])
tags

,UNIQUE_ID,Value
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
21245,21245,0
21246,21246,0
21247,21247,1
21248,21248,0


In [13]:
tags["UNIQUE_ID"]=tags["UNIQUE_ID"].map(d_swap)
tags=tags.rename(columns={'UNIQUE_ID' : 'ID' , "Value":"label"})

In [14]:
tags

,ID,label
0,7510,0
1,10116,0
2,259,0
3,20870,0
4,3090,0
...,...,...
21245,2343,0
21246,4206,0
21247,20393,1
21248,8392,0


In [15]:
tags.to_csv(outfile_path+"GRU_ODE_death_tags.csv",index=False)

In [16]:
tags.columns

Index(['ID', 'label'], dtype='object')

## Covs process

In [17]:
covs

,UNIQUE_ID,004,008,009,011,012,013,021,027,031,...,997,998,999,V45,V50,V55,V56,V58,V59,V71
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21245,21245,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21246,21246,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21247,21247,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21248,21248,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
covs["UNIQUE_ID"]=covs["UNIQUE_ID"].map(d_swap)
covs=covs.rename(columns={'UNIQUE_ID' : 'ID'})

In [19]:
covs

,ID,004,008,009,011,012,013,021,027,031,...,997,998,999,V45,V50,V55,V56,V58,V59,V71
0,7510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10116,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,259,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20870,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3090,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21245,2343,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21246,4206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21247,20393,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21248,8392,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
covs.to_csv(outfile_path+"GRU_ODE_covariates.csv",index=False)

## sapsii


In [21]:
saps

,Unnamed: 0,subject_id,hadm_id,icustay_id,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,...,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score,HADM_ID,SUM_score,X,PROB
0,1,27513,163557,200003,30,0.106398,7,4.0,5.0,0.0,...,0.0,3.0,0.0,0.0,0,8,163557,15.0,-3.893052,0.980024
1,3,20707,129310,200007,18,0.029295,7,0.0,5.0,0.0,...,0.0,0.0,NaN,0.0,0,6,129310,5.0,-5.608037,0.996345
2,8,9514,127229,200014,43,0.305597,18,2.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0,8,127229,17.0,-3.628210,0.974124
3,16,76603,179633,200024,52,0.507019,18,4.0,5.0,0.0,...,1.0,3.0,NaN,0.0,0,6,179633,28.0,-2.341969,0.912294
4,17,19167,164161,200025,29,0.096698,7,0.0,5.0,0.0,...,0.0,0.0,NaN,0.0,0,8,164161,14.0,-4.031103,0.982555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22549,61523,27349,126991,299986,26,0.071716,16,2.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0,8,126991,2.0,-6.520274,0.998529
22550,61527,13620,169431,299993,12,0.013110,0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,8,169431,4.0,-5.863529,0.997167
22551,61528,10718,177406,299994,37,0.196366,15,2.0,5.0,0.0,...,0.0,0.0,NaN,0.0,0,6,177406,16.0,-3.758903,0.977222
22552,61530,69587,158288,299998,49,0.437547,18,2.0,5.0,0.0,...,0.0,0.0,NaN,0.0,9,6,158288,16.0,-3.758903,0.977222


In [22]:
saps["icustay_id"].nunique(), saps["hadm_id"].nunique(), saps["HADM_ID"].nunique()

(22554, 21240, 21240)

In [23]:
df = saps.groupby("hadm_id")["icustay_id"].nunique()
ndup_ids=list(df[df==1].index) #index of patients with only one visit.
saps_1=saps[~saps["hadm_id"].isin(ndup_ids)] #filter out the patients with more than one visit
print(len(saps_1.index))
print(len(df), sum(df.values), min(df), max(df))
saps_1.sort_values("hadm_id")

2550
21240 22554 1 4


,Unnamed: 0,subject_id,hadm_id,icustay_id,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,...,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score,HADM_ID,SUM_score,X,PROB
3388,9399,50417,100223,215164,44,0.326364,18,4.0,5.0,0.0,...,1.0,0.0,0.0,0.0,0,6,100223,20.0,-3.253407,0.962795
2542,7062,50417,100223,211396,61,0.700098,18,2.0,5.0,0.0,...,0.0,5.0,0.0,0.0,0,6,100223,37.0,-1.409163,0.803634
14105,38646,49079,100234,262878,65,0.769396,18,4.0,5.0,0.0,...,0.0,3.0,NaN,0.0,0,6,100234,41.0,-1.014570,0.733913
12765,34960,49079,100234,256814,41,0.266087,18,2.0,5.0,0.0,...,1.0,3.0,0.0,0.0,0,6,100234,17.0,-3.628210,0.974124
14122,38704,45797,100406,262970,25,0.064649,15,2.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0,8,100406,2.0,-6.520274,0.998529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,6932,9057,199947,211179,39,0.229592,18,2.0,5.0,0.0,...,0.0,0.0,NaN,0.0,0,8,199947,13.0,-4.173596,0.984837
19070,52069,9057,199947,284602,33,0.140051,18,2.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0,8,199947,7.0,-5.173789,0.994369
17095,46676,22711,199952,275802,30,0.106398,18,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,0,6,199952,6.0,-5.380633,0.995416
19613,53537,22711,199952,286980,24,0.058153,18,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,0,6,199952,0.0,-7.763100,0.999575


icustay_idとhadm_idの分離がうまくいっていない。要改修であるが、さしあたり最もicustay_idが小さいもののみを残す。

In [24]:
df = saps.groupby("hadm_id")["icustay_id"].min()
ndup_ids=list(df.values) #index of patients with only one visit.
saps_2=saps[saps["icustay_id"].isin(ndup_ids)] #filter out the patients with more than one visit
saps_2.sort_values("hadm_id")

,Unnamed: 0,subject_id,hadm_id,icustay_id,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,...,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score,HADM_ID,SUM_score,X,PROB
4009,11096,23018,100007,217937,25,0.064649,15,2.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0,8,100007,2.0,-6.520274,0.998529
12075,33063,533,100009,253656,36,0.181020,12,2.0,5.0,NaN,...,0.0,0.0,NaN,0.0,0,8,100009,16.0,-3.758903,0.977222
16014,43798,55853,100010,271147,37,0.196366,7,2.0,13.0,0.0,...,0.0,0.0,NaN,0.0,9,6,100010,15.0,-3.893052,0.980024
3275,9057,87977,100011,214619,21,0.041754,0,4.0,5.0,0.0,...,0.0,0.0,NaN,0.0,0,6,100011,15.0,-3.893052,0.980024
3941,10899,68591,100016,217590,40,0.247443,7,0.0,5.0,0.0,...,0.0,0.0,NaN,7.0,0,6,100016,27.0,-2.450659,0.920610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,958,42131,199967,201521,27,0.079390,18,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0,6,199967,3.0,-6.159726,0.997892
3489,9663,26198,199976,215595,32,0.128048,16,2.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0,8,199976,8.0,-4.982647,0.993191
17214,47017,28549,199987,276369,26,0.071716,15,0.0,5.0,0.0,...,0.0,0.0,NaN,0.0,0,6,199987,5.0,-5.608037,0.996345
16162,44150,20785,199993,271752,23,0.052195,12,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0,6,199993,5.0,-5.608037,0.996345


In [25]:
hadm_uniq["ID"]=hadm_uniq["UNIQUE_ID"].map(d_swap)
hadm_uniq["ID"].nunique(),hadm_uniq["UNIQUE_ID"].nunique() 

(21250, 21250)

In [26]:
d_adm_uniq = dict(zip(hadm_uniq["HADM_ID"].values,hadm_uniq["UNIQUE_ID"].values))

In [27]:
saps_2["UNIQUE_ID"] = saps_2["HADM_ID"].map(d_adm_uniq)
saps_2["ID"] = saps_2["UNIQUE_ID"].map(d_swap)
saps_2

,Unnamed: 0,subject_id,hadm_id,icustay_id,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,...,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score,HADM_ID,SUM_score,X,PROB,UNIQUE_ID,ID
0,1,27513,163557,200003,30,0.106398,7,4.0,5.0,0.0,...,0.0,0.0,0,8,163557,15.0,-3.893052,0.980024,14771,10444
1,3,20707,129310,200007,18,0.029295,7,0.0,5.0,0.0,...,NaN,0.0,0,6,129310,5.0,-5.608037,0.996345,8447,3356
2,8,9514,127229,200014,43,0.305597,18,2.0,5.0,0.0,...,0.0,0.0,0,8,127229,17.0,-3.628210,0.974124,15179,16859
3,16,76603,179633,200024,52,0.507019,18,4.0,5.0,0.0,...,NaN,0.0,0,6,179633,28.0,-2.341969,0.912294,18678,20933
4,17,19167,164161,200025,29,0.096698,7,0.0,5.0,0.0,...,NaN,0.0,0,8,164161,14.0,-4.031103,0.982555,12072,17315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22546,61519,265,101608,299976,25,0.064649,7,4.0,5.0,0.0,...,NaN,0.0,0,0,101608,18.0,-3.500600,0.970705,10792,12435
22547,61520,42728,142693,299979,37,0.196366,12,2.0,5.0,0.0,...,NaN,0.0,9,6,142693,10.0,-4.635159,0.990389,19372,20928
22548,61521,17532,129682,299981,40,0.247443,12,0.0,13.0,0.0,...,NaN,0.0,0,6,129682,22.0,-3.015299,0.953261,16314,19174
22550,61527,13620,169431,299993,12,0.013110,0,4.0,0.0,0.0,...,0.0,0.0,0,8,169431,4.0,-5.863529,0.997167,11785,10464


In [28]:
temp = saps_2.iloc[:,range(4,21)]
temp = pd.concat([temp,saps_2.iloc[:,range(22,25)]], axis=1)
temp = pd.concat([saps_2.loc[:,["ID","UNIQUE_ID"]],temp], axis=1)
saps_3 = temp.sort_values("ID")
saps_3

,ID,UNIQUE_ID,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,temp_score,pao2fio2_score,uo_score,...,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score,SUM_score,X,PROB
2067,0,12623,67,0.799323,0,4.0,13.0,0.0,11.0,0.0,...,3.0,1.0,0.0,0.0,26.0,0,6,61.0,0.847766,0.299902
17713,1,13781,41,0.266087,12,2.0,0.0,0.0,9.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0,6,23.0,-2.899163,0.947805
19682,2,1326,55,0.575439,18,4.0,5.0,0.0,NaN,4.0,...,0.0,0.0,3.0,9.0,0.0,0,6,31.0,-2.022715,0.883161
7489,3,8484,38,0.212560,7,0.0,0.0,0.0,6.0,4.0,...,3.0,0.0,0.0,4.0,0.0,0,8,23.0,-2.899163,0.947805
13245,4,16753,37,0.196366,18,4.0,0.0,0.0,6.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0,6,13.0,-4.173596,0.984837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18008,21245,6845,29,0.096698,12,0.0,5.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0,6,11.0,-4.474700,0.988735
3608,21246,839,23,0.052195,7,2.0,5.0,0.0,NaN,0.0,...,3.0,0.0,0.0,NaN,0.0,0,6,10.0,-4.635159,0.990389
15221,21247,20187,73,0.870860,7,4.0,13.0,0.0,6.0,11.0,...,0.0,1.0,5.0,9.0,5.0,0,6,60.0,0.757852,0.319113
8193,21248,19889,43,0.305597,16,0.0,5.0,0.0,6.0,4.0,...,3.0,0.0,3.0,0.0,0.0,0,0,27.0,-2.450659,0.920610


In [29]:
saps_3.to_csv(outfile_path+"GRU_ODE_sapsii.csv",index=False)

IDが歯抜けなのの要注意。HADM_IDではなくICUSTAY_IDで初めから監理したほうが、sapsiiがないデータは除くなりしてから学習データを作成したほうが整合性は取れている。

In [30]:
saps_3.shape, saps_3["ID"].nunique(), saps_3["UNIQUE_ID"].nunique()

((21240, 22), 21240, 21240)